In [17]:
import pandas as pd

file_path = r"/content/Glassdoor_Salary.csv"

# Load the CSV file
dataset = pd.read_csv(file_path)

# Display the first few rows of the dataset
print(dataset.head())

                   Job Title              Salary Estimate  \
0             Data Scientist   $53K-$91K (Glassdoor est.)   
1  Healthcare Data Scientist  $63K-$112K (Glassdoor est.)   
2             Data Scientist   $80K-$90K (Glassdoor est.)   
3             Data Scientist   $56K-$97K (Glassdoor est.)   
4             Data Scientist  $86K-$143K (Glassdoor est.)   

                                     Job Description  Rating  \
0  Data Scientist\nLocation: Albuquerque, NM\nEdu...     3.8   
1  What You Will Do:\n\nI. General Summary\n\nThe...     3.4   
2  KnowBe4, Inc. is a high growth information sec...     4.8   
3  *Organization and Job ID**\nJob ID: 310709\n\n...     3.8   
4  Data Scientist\nAffinity Solutions / Marketing...     2.9   

                                 Company Name         Location  \
0                      Tecolote Research\n3.8  Albuquerque, NM   
1  University of Maryland Medical System\n3.4    Linthicum, MD   
2                                KnowBe4\n4.8   Cl

In [18]:
# List of unwanted columns to delete
columns_to_delete = [
    "Salary Estimate",
    "Job Description",
    "Sector",
    "Competitors",
    "hourly",
    "employer_provided",
    "company_txt",
    "job_state",
    "same_state"
]

# Drop the columns
dataset = dataset.drop(columns=columns_to_delete)

# Display the first few rows of the updated dataset
print(dataset.head())

                   Job Title  Rating  \
0             Data Scientist     3.8   
1  Healthcare Data Scientist     3.4   
2             Data Scientist     4.8   
3             Data Scientist     3.8   
4             Data Scientist     2.9   

                                 Company Name         Location  \
0                      Tecolote Research\n3.8  Albuquerque, NM   
1  University of Maryland Medical System\n3.4    Linthicum, MD   
2                                KnowBe4\n4.8   Clearwater, FL   
3                                   PNNL\n3.8     Richland, WA   
4                     Affinity Solutions\n2.9     New York, NY   

     Headquarters                    Size  Founded   Type of ownership  \
0      Goleta, CA   501 to 1000 employees     1973   Company - Private   
1   Baltimore, MD        10000+ employees     1984  Other Organization   
2  Clearwater, FL   501 to 1000 employees     2010   Company - Private   
3    Richland, WA  1001 to 5000 employees     1965          Govern

In [19]:
# Check for duplicate records
duplicate_records = dataset.duplicated()

# Display the number of duplicate records
print(f"Number of duplicate records: {duplicate_records.sum()}")

# Remove duplicate records
dataset = dataset.drop_duplicates()

# Display the shape of the dataset after removing duplicates
print(f"Dataset shape after removing duplicates: {dataset.shape}")

Number of duplicate records: 275
Dataset shape after removing duplicates: (467, 19)


In [20]:
# Splitting 'Location' column into 'City' and 'State'
dataset[['City', 'State']] = dataset['Location'].str.split(',', n=1, expand=True)

# Cleaning extra spaces in 'City' and 'State'
dataset['City'] = dataset['City'].str.strip()
dataset['State'] = dataset['State'].str.strip()

# Fill missing values in 'City' or 'State' if any
dataset['City'] = dataset['City'].fillna('Unknown')
dataset['State'] = dataset['State'].fillna('Unknown')

# Deleting the 'Location' column
dataset = dataset.drop(columns=['Location'])

# Display the first few rows of the updated dataset
print(dataset.head())

                   Job Title  Rating  \
0             Data Scientist     3.8   
1  Healthcare Data Scientist     3.4   
2             Data Scientist     4.8   
3             Data Scientist     3.8   
4             Data Scientist     2.9   

                                 Company Name    Headquarters  \
0                      Tecolote Research\n3.8      Goleta, CA   
1  University of Maryland Medical System\n3.4   Baltimore, MD   
2                                KnowBe4\n4.8  Clearwater, FL   
3                                   PNNL\n3.8    Richland, WA   
4                     Affinity Solutions\n2.9    New York, NY   

                     Size  Founded   Type of ownership  \
0   501 to 1000 employees     1973   Company - Private   
1        10000+ employees     1984  Other Organization   
2   501 to 1000 employees     2010   Company - Private   
3  1001 to 5000 employees     1965          Government   
4     51 to 200 employees     1998   Company - Private   

                   

In [21]:
# Function to clean the "Size" column, by considering average value
def clean_size(value):
    if value in ["-1", "Unknown"]:
        return "Unknown"  # Replace both "-1" and "Unknown" with "Unknown"
    elif "to" in value:
        # For ranges like "1 to 50", calculate the mid value
        low, high = map(int, value.replace("employees", "").strip().split(" to "))
        return (low + high) // 2
    elif "+" in value:
        # For values like "10000+ employees", extract the number before "+"
        return int(value.replace("employees", "").replace("+", "").strip())
    return value

# Apply the function to the "Size" column
dataset['Size'] = dataset['Size'].apply(clean_size)

# Display unique cleaned values
print(dataset['Size'].unique())

[750 10000 3000 125 350 7500 25 'Unknown']


In [22]:
# Function to clean the "Type of ownership" column
def clean_ownership(ownership):
    if ownership in ["-1", "Unknown", "Other Organization"]:
        return "Unknown"
    elif ownership in ["College / University", "School / School District"]:
        return "School or College"
    else:
        return ownership  # Keep other values as they are

# Apply the function to the "Type of ownership" column
dataset['Type of ownership'] = dataset['Type of ownership'].apply(clean_ownership)

# Display the unique values in the cleaned "Type of ownership" column
print(dataset['Type of ownership'].unique())


['Company - Private' 'Unknown' 'Government' 'Company - Public' 'Hospital'
 'Subsidiary or Business Segment' 'Nonprofit Organization'
 'School or College']


In [23]:
# Check for data types of each column
print("\nDataset Information:")
print(dataset.info())


Dataset Information:
<class 'pandas.core.frame.DataFrame'>
Index: 467 entries, 0 to 735
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Job Title          467 non-null    object 
 1   Rating             467 non-null    float64
 2   Company Name       467 non-null    object 
 3   Headquarters       467 non-null    object 
 4   Size               467 non-null    object 
 5   Founded            467 non-null    int64  
 6   Type of ownership  467 non-null    object 
 7   Industry           467 non-null    object 
 8   Revenue            467 non-null    object 
 9   min_salary         467 non-null    int64  
 10  max_salary         467 non-null    int64  
 11  avg_salary         467 non-null    float64
 12  age                467 non-null    int64  
 13  python_yn          467 non-null    int64  
 14  R_yn               467 non-null    int64  
 15  spark              467 non-null    int64  
 16  aws      

In [24]:
# Check for missing values in the dataset
missing_values = dataset.isnull().sum()

# Display columns with missing values and their count
print("Missing Values in Each Column:")
print(missing_values[missing_values > 0])

Missing Values in Each Column:
Series([], dtype: int64)


In [25]:
# File path to save the updated dataset
output_file_path = r"/content/Cleaned_Glassdoor_Salary.csv"

# Save the dataset to the specified path
dataset.to_csv(output_file_path, index=False)